In [2]:
from itertools import product

In [3]:
p1 = list(range(1, 100))
p2 = list(range(1, 10))
p3 = list(range(1, 20))

In [4]:
params_combinations = list(product(p1, p2, p3))

In [5]:
len(params_combinations)

16929

In [6]:
def foo(a, b, c):
    if a % 3:
        res = a ** b + c
    elif a % 2:
        res = a * b
    else:
        res = c ** (a + b)
    return res

In [16]:
%%timeit -n5
for params in params_combinations:
    foo(*params)

8.92 ms ± 628 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [8]:
results1 = {}
for params in params_combinations:
    results1[params] = foo(*params)

In [14]:
def mp_foo(abc):
    return foo(*abc)

In [10]:
import multiprocess

In [11]:
%%timeit -n5
with multiprocess.Pool(2) as pool:
    pool.map(
        mp_foo,
        params_combinations
    )

NameError: name 'foo' is not defined

In [12]:
from concurrent.futures import ThreadPoolExecutor

In [20]:
%%timeit -n5
with ThreadPoolExecutor(2) as executor:
    executor.map(
        foo,
        params_combinations
    )

539 ms ± 71.4 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [ ]:
from threading import Thread
from threading import Semaphore

def foo_mp_write(semaphore, abc, results2):
    with semaphore:
        a, b, c = abc
        results2[abc] = foo(a, b, c)

results2 = {}
semaphore = Semaphore(200)
for params in params_combinations:
    a, b, c = params
    worker = Thread(target=foo_mp_write, args=(semaphore, params, results2))
    worker.start()

In [ ]:
assert results1 == results2